In [1]:
path = './multilabel/stackoverflow_perguntas.csv'

In [2]:
import pandas as pd
df = pd.read_csv(path)

In [3]:
df.shape

(5408, 2)

In [4]:
df.sample(10)

,Perguntas,Tags
1602,"Tenho um sistema todo em ajax, portando quando...",html
5166,Observei que a página tem o seguinte código pa...,html angular
600,Peguei um exemplo de um slider de uma pergunta...,html
4556,Preciso fazer um formulário desaparecer gradat...,angular
1797,É possível percorrer um textarea para detectar...,jquery html
301,Tenho o seguinte input file que faz o upload d...,html angular
835,Estou tentando trazer um valor que ao clicar e...,jquery
666,Tenho essa classe no meu CSS: CODE Ela não ...,jquery
4945,estou tentando obter um json de retorno da fun...,angular
749,Recentemente fiz a pergunta Porque devemos usa...,jquery


In [5]:
df['Tags'].unique()

array(['node.js', 'jquery', 'html', 'html angular ', 'html ', 'angular',
       'angular ', 'jquery html  ', 'jquery ', 'jquery html',
       'jquery html ', 'html angular', 'angular node.js ', 'html  ',
       'jquery html angular', 'node.js ', 'html jquery', 'html jquery ',
       'jquery angular  ', 'html node.js', 'jquery  ', 'angular node.js',
       'jquery angular', 'html node.js ', 'jquery node.js ', 'angular  ',
       'jquery angular ', 'jquery html angular ', 'node.js html ',
       ' node.js', 'node.js html', 'html angular  ', 'jquery node.js',
       'angular html', 'html angular  node.js', 'jquery html node.js',
       'html angular node.js'], dtype=object)

In [6]:
#vamos criar um modo de converter de string para inteiros binarios
# Removendo espaços em excesso nas strings
df['Tags'] = df['Tags'].str.strip()

# Separando as tags em colunas binárias
tags_dummies = df['Tags'].str.get_dummies(sep=' ')

# Adicionando as colunas resultantes ao DataFrame original
df = pd.concat([df, tags_dummies], axis=1)

In [7]:
df.sample(3)

,Perguntas,Tags,angular,html,jquery,node.js
4592,Tenho uma aplicação frontend Angular consumind...,angular,1,0,0,0
4118,Estou com problema na migração do ionic beta.1...,angular,1,0,0,0
2024,"Tenho o seguinte código, preciso verificar o s...",angular,1,0,0,0


In [8]:
df['juncao'] = list(zip(df['angular'], df['html'], df['jquery'], df['node.js']))

In [9]:
df.sample(10)

,Perguntas,Tags,angular,html,jquery,node.js,juncao
4918,Olá! Estou tentando alterar uma máscara de um...,jquery,0,0,1,0,"(0, 0, 1, 0)"
2063,tenho um campo para inserir telefone com opçao...,jquery,0,0,1,0,"(0, 0, 1, 0)"
5320,Sabendo que a partir deste ano o crawler do go...,html angular,1,1,0,0,"(1, 1, 0, 0)"
3354,Estou tentando passar para o meu código javasc...,html,0,1,0,0,"(0, 1, 0, 0)"
2986,Exemplo var numero = 3500; Quero que reto...,html,0,1,0,0,"(0, 1, 0, 0)"
5389,"Tenho minha seguinte tela : O menu lateral, ...",angular,1,0,0,0,"(1, 0, 0, 0)"
2561,"Tenho um uma página, onde devo realizar traduç...",jquery,0,0,1,0,"(0, 0, 1, 0)"
3720,Estou precisando fazer uma máscara de valores ...,angular,1,0,0,0,"(1, 0, 0, 0)"
35,"Sem muita enrolação, estou preso no seguinte p...",html,0,1,0,0,"(0, 1, 0, 0)"
1562,Criei o Seguinte código SP Services porem o me...,jquery,0,0,1,0,"(0, 0, 1, 0)"


In [10]:
# quais classificadores usar para multilabel

In [11]:
# hora do modelo

In [12]:
from sklearn.model_selection import train_test_split
X = df['Perguntas']
y = df['juncao']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [13]:
# TF-IDF
# texto para numeros - A principal característica do TF-IDF é ser uma pontuação proporcional à frequência da palavra
# no texto e equilibrada pela frequência no corpus, ou seja, palavras que se repetem muito tendem a ter pontuações 
# menores e são menos relevantes no processo de classificação.

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000,max_df=0.85) # maximo  vetor de palavras e remover palavras com muita frequencia

In [15]:
vectorizer.fit(df.Perguntas)

TfidfVectorizer(max_df=0.85, max_features=5000)

In [16]:
vectorizer.get_feature_names_out()

array(['00', '000', '01', ..., 'único', 'únicos', 'útil'], dtype=object)

In [17]:
X_train_tfidf = vectorizer.transform(X_train)
X_test_tfidf  = vectorizer.transform(X_test)

In [18]:
# relevancia binaria onevsrestclassifier

In [19]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression()
classificador = OneVsRestClassifier(lg)

In [20]:
classificador

OneVsRestClassifier(estimator=LogisticRegression())

In [26]:
import numpy as np 
y_train_array = np.asarray(list(y_train))
y_test_array = np.asarray(list(y_test))

In [27]:
classificador.fit(X_train_tfidf, y_train_array)

OneVsRestClassifier(estimator=LogisticRegression())

In [29]:
classificador.score(X_test_tfidf,y_test_array)

0.3927170868347339

In [32]:
scr = classificador.score(X_test_tfidf,y_test_array)

In [34]:
# hamming loss para multilabel
# A principal diferença é que o Hamming loss avalia os acertos de cada label individualmente, 
# enquanto a acurácia avalia a combinação das labels, ou seja, será considerado correto apenas se o 
# classificador acertar todas as labels de uma determinada instância. Para o Hamming Loss, 
# quanto mais próximo de 0 melhor o resultado, enquanto para a acurácia o melhor resultado é o mais próximo de 1.

In [39]:
from sklearn.metrics import hamming_loss

previsao_onevsrest = classificador.predict(X_test_tfidf)
hamming_loss_onevsrest = hamming_loss(y_test_array, previsao_onevsrest)
print("Hamming Loss {0: .2f}".format(hamming_loss_onevsrest))

Hamming Loss  0.19
